# SQL avanzado y diseño de bases de datos
### **Ingeniería de datos**
**Profesor: Domagoj Vrgoč**

### Introducción

En este notebook repasaremos los conceptos avanzados de SQL cómo joins, agregación, y agrupamiento.

### Requisitos

Para esta actividad, así como en las siguientes actividades de SQL vamos a utilizar *Google colab* (https://colab.research.google.com), que es un entorno virtual permitiendo armar un servidor de bases de datos, y conectarse con este servidor. Para la conexión ocuparemos la herramienta llamada Jupyter Notebooks. Esta herramienta permite conectarse con un servidor SQL de la misma manera cómo hacerlo a través de la consola en un servidor local. 

El motor de bases de datos que ocuparemos en este curso se llama PostgreSQL, y uno siempre puede instalarlo localmente en su computador. Idea de ocupar Google colab es saltarse este paso, y no tener problemas con instalar, habilitar, o correr un motor de bases de datos.

Por lo tanto, para una actividad de SQL, en este curso siempre ocuparemos Jupyter Notebooks con Google colab. Para esto, se les entregará un archivo con extensión .ipynb, cual hay que subir a la plataforma Google Colab. Al inicio del tutorial mostraremos cómo funciona este proceso.

### Outline

En esta actividad aprenderemos a:

- Habilitar un servidor de PostgreSQL en Google colab con un Jupyter notebook.
- Definir esquema de una tabla relacional.
- Insertar datos.
- Hacer joins complejos
- Agrupar a los resultados de una consulta
- Agregar resultados

### Esquema

Para esta actividad vamos a trabajar con el siguiente esquema:

- `Productos(pid, nombre, precio, stock)`

- `Usuarios(uid, nombre, correo)`

- `Compras(cid, uid, valor)`

- `ProductosComprados(cid, pid, cantidad)`

Que corresponde a películas, actores, y la información de los roles interpretados por una actor en una película.

Las llaves en nuestro caso son:
1. `pid`, para `Productos`
2. `uid` para `Usuarios`
3. `cid` para `Compras`
4. `(cid,pid)` para `ProductosComprados`.

## Tutorial

Lo primero que hay que hacer es subir este notebook a https://colab.research.google.com

### Iniciar el servidor

Para iniciar el servidor virtual, *instalar* la base de datos postgres debe correr el siguiente bloque:

In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# pin SQLAlchemy version for colab compatibility
!pip install SQLAlchemy==1.4.46
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,938 kB]
Hit:13 htt

'Connected: @postgres'

### Definiendo nuestro esquema

In [2]:
%%sql

CREATE TABLE IF NOT EXISTS Productos(
    pid int PRIMARY KEY,
    nombre varchar(20),
    precio float,
    stock int
);

CREATE TABLE IF NOT EXISTS Usuarios(
    uid int PRIMARY KEY,
    nombre varchar(40),
    correo varchar(40)
);

CREATE TABLE IF NOT EXISTS Compras(
    cid int PRIMARY KEY,
    uid int,
    valor float
);

CREATE TABLE IF NOT EXISTS ProductosComprados(
    cid int,
    pid int,
    cantidad int,
    PRIMARY KEY (cid,pid)
);



 * postgresql+psycopg2://@/postgres


""


### Insertar datos en la tabla

Para revisar el contenido de una tabla, podemos correr la consulta `SELECT * FROM NombreTabla` (más de esto en la siguiente clase).

Si ejecutamos la consulta `SELECT * FROM Peliculas` notaremos que el resultado es vacío. Esto ocurre porque todavía no hemos insertado ningun dato a la tabla.

In [3]:
%sql SELECT * FROM Productos;

 * postgresql+psycopg2://@/postgres


""


In [4]:
%%sql 
INSERT INTO Productos VALUES(1, 'iPad 78895', 900000,21);
INSERT INTO Productos VALUES(2, 'iPad 4', 700000,2);
INSERT INTO Productos VALUES(3, 'Dell XPS 13 Plus', 1700000,12);

SELECT * FROM Productos;

 * postgresql+psycopg2://@/postgres


,pid,nombre,precio,stock
0,1,iPad 78895,900000.0,21
1,2,iPad 4,700000.0,2
2,3,Dell XPS 13 Plus,1700000.0,12


Para las otras tablas:

In [5]:
%%sql 

INSERT INTO Usuarios VALUES(1, 'Juan Reutter', 'jreutter@dcc.uc.cl');
INSERT INTO Usuarios VALUES(2, 'Adrián Soto', 'asoto@uai.cl');
INSERT INTO Usuarios VALUES(3, 'criveros22@dcc.uc.cl', 'jreutter@dcc.uc.cl');

INSERT INTO Compras VALUES(1,2,2300000);
INSERT INTO Compras VALUES(2,3,1700000);
INSERT INTO Compras VALUES(3,2,1700000);
INSERT INTO Compras VALUES(4,2,1600000);

INSERT INTO ProductosComprados VALUES(1,2,2);
INSERT INTO ProductosComprados VALUES(1,1,1);
INSERT INTO ProductosComprados VALUES(2,3,1);
INSERT INTO ProductosComprados VALUES(3,3,1);
INSERT INTO ProductosComprados VALUES(4,1,1);
INSERT INTO ProductosComprados VALUES(4,2,1);

 * postgresql+psycopg2://@/postgres


""


#Joins

Nuestra primera consulta pide retornar los ids y valores de las compras hechas por Adrián Soto, ordenadas por valor.


In [6]:
%%sql

SELECT Compras.cid, Compras.valor
FROM Usuarios, Compras
WHERE Usuarios.nombre = 'Adrián Soto' AND
      Usuarios.uid = Compras.uid
ORDER BY Compras.valor;

 * postgresql+psycopg2://@/postgres


,cid,valor
0,4,1600000.0
1,3,1700000.0
2,1,2300000.0


La siguiente pregunta nos pide devolver los nombres de todos los productos comprados por Adrián Soto, eliminando a los duplicados. Esta consulta involucra las 4 tablas de nuestro esquema.

In [8]:
%%sql

SELECT DISTINCT Productos.nombre, Productos.precio
FROM Usuarios, Compras, ProductosComprados, Productos
WHERE Usuarios.nombre = 'Adrián Soto' AND
      Usuarios.uid = Compras.uid AND
      Compras.cid = ProductosComprados.cid AND
      ProductosComprados.pid = Productos.pid
ORDER BY Productos.nombre;

 * postgresql+psycopg2://@/postgres


,nombre,precio
0,Dell XPS 13 Plus,1700000.0
1,iPad 4,700000.0
2,iPad 4,700000.0
3,iPad 78895,900000.0
4,iPad 78895,900000.0


Observen que sin la palabra clave DISTINCT, recibiremos muchas repeticiones:

In [9]:
%%sql

SELECT Productos.nombre, Productos.precio
FROM Usuarios, Compras, ProductosComprados, Productos
WHERE Usuarios.nombre = 'Adrián Soto' AND
      Usuarios.uid = Compras.uid AND
      Compras.cid = ProductosComprados.cid AND
      ProductosComprados.pid = Productos.pid
ORDER BY Productos.nombre;

 * postgresql+psycopg2://@/postgres


,nombre,precio
0,Dell XPS 13 Plus,1700000.0
1,iPad 4,700000.0
2,iPad 4,700000.0
3,iPad 78895,900000.0
4,iPad 78895,900000.0


#Agregación y agrupamiento

Número de productos distintos por compra (repeticiones no cuentan):

In [19]:
%%sql 

SELECT cid, COUNT(*)
FROM ProductosComprados
GROUP BY cid;

 * postgresql+psycopg2://@/postgres


,cid,count
0,3,1
1,4,2
2,2,1
3,1,2


Número total de productos en cada compra:

In [25]:
%%sql 

SELECT cid, SUM(cantidad)
FROM ProductosComprados
GROUP BY cid;

 * postgresql+psycopg2://@/postgres


,cid,sum
0,3,1
1,4,2
2,2,1
3,1,3


Número de productos (total) en cada compra. Nos quedamos solo con compras con más de dos items.

In [21]:
%%sql

SELECT cid, SUM(cantidad)
FROM ProductosComprados
GROUP BY cid
HAVING SUM(cantidad)>2;

 * postgresql+psycopg2://@/postgres


,cid,sum
0,1,3


Número de compras por usuario:

In [24]:
%%sql

SELECT Usuarios.nombre, COUNT(*)
FROM Usuarios, Compras
WHERE Usuarios.uid = Compras.uid
GROUP BY  Usuarios.nombre

 * postgresql+psycopg2://@/postgres


,nombre,count
0,criveros22@dcc.uc.cl,1
1,Adrián Soto,3


Para cada usuario el precio de producto más carro qué compró:

In [27]:
%%sql

SELECT Usuarios.nombre, MAX(Productos.precio)
FROM Usuarios, Compras, ProductosComprados, Productos
WHERE Usuarios.uid = Compras.uid AND
      Compras.cid = ProductosComprados.cid AND
      ProductosComprados.pid = Productos.pid
GROUP BY Usuarios.nombre;

 * postgresql+psycopg2://@/postgres


,nombre,max
0,criveros22@dcc.uc.cl,1700000.0
1,Adrián Soto,1700000.0


Para cada usuario, y cada una de sus compras con dos o más items, el precio de producto más carro comprado ene esta compra:

In [28]:
%%sql

SELECT Usuarios.nombre, Compras.cid, MAX(Productos.precio)
FROM Usuarios, Compras, ProductosComprados, Productos
WHERE Usuarios.uid = Compras.uid AND
      Compras.cid = ProductosComprados.cid AND
      ProductosComprados.pid = Productos.pid
GROUP BY Usuarios.nombre, Compras.cid
HAVING SUM(ProductosComprados.cantidad)>=2;

 * postgresql+psycopg2://@/postgres


,nombre,cid,max
0,Adrián Soto,1,900000.0
1,Adrián Soto,4,900000.0


#Consultas anidadas

Nombre del producto más carro comprado por cada usuario:

In [31]:
%%sql

SELECT MaxPrecios.nombre, Productos.nombre
FROM Productos,  (SELECT Usuarios.nombre AS nombre, MAX(Productos.precio) AS maximo
                  FROM Usuarios, Compras, ProductosComprados, Productos
                  WHERE Usuarios.uid = Compras.uid AND
                        Compras.cid = ProductosComprados.cid AND
                        ProductosComprados.pid = Productos.pid
                  GROUP BY Usuarios.nombre) AS MaxPrecios
WHERE MaxPrecios.maximo = Productos.precio;

 * postgresql+psycopg2://@/postgres


,nombre,nombre
0,Adrián Soto,Dell XPS 13 Plus
1,criveros22@dcc.uc.cl,Dell XPS 13 Plus


Solución 100% correcta:

In [33]:
%%sql

SELECT MaxPrecios.nombre, Productos.nombre
FROM Productos,  (SELECT Usuarios.nombre AS nombre, Usuarios.uid AS uid, MAX(Productos.precio) AS maximo
                  FROM Usuarios, Compras, ProductosComprados, Productos
                  WHERE Usuarios.uid = Compras.uid AND
                        Compras.cid = ProductosComprados.cid AND
                        ProductosComprados.pid = Productos.pid
                  GROUP BY Usuarios.nombre, Usuarios.uid) AS MaxPrecios, Compras, ProductosComprados
WHERE MaxPrecios.maximo = Productos.precio AND
      MaxPrecios.uid = Compras.uid AND
      Compras.cid = ProductosComprados.cid AND
      ProductosComprados.pid = Productos.pid;

 * postgresql+psycopg2://@/postgres


,nombre,nombre
0,criveros22@dcc.uc.cl,Dell XPS 13 Plus
1,Adrián Soto,Dell XPS 13 Plus
